Trains a simple convnet on the MNIST dataset.

In [1]:
import numpy as np

from kerasy.datasets import mnist
from kerasy.models import Sequential
from kerasy.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from kerasy.utils import CategoricalEncoder

In [2]:
# Datasets Parameters.
num_classes = 10
n_samples = 1_000

# Training Parameters.
batch_size = 16
epochs = 20
keep_prob1 = 0.75
keep_prob2 = 0.5

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28

In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train = np.expand_dims(x_train, axis=-1)
x_test  = np.expand_dims(x_test,  axis=-1)
input_shape = (img_rows, img_cols, 1)

In [6]:
x_train = x_train[:n_samples]
y_train = y_train[:n_samples]
x_test = x_test[:n_samples]
y_test = y_test[:n_samples]

In [7]:
x_train = x_train.astype('float64')
x_test = x_test.astype('float64')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (1000, 28, 28, 1)
1000 train samples
1000 test samples


In [8]:
# convert class vectors to binary class matrices
encoder = CategoricalEncoder()
y_train = encoder.to_onehot(y_train, num_classes)
y_test  = encoder.to_onehot(y_test, num_classes)

Dictionaly for Encoder is already made.


In [9]:
model = Sequential()
model.add(Input(input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(keep_prob=keep_prob1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(keep_prob=keep_prob2))
model.add(Dense(num_classes, activation='softmax'))

In [10]:
model.compile(
    optimizer='adagrad', 
    loss='categorical_crossentropy', 
    metrics=['categorical_accuracy']
)

/Users/iwasakishuto/Github/portfolio/Kerasy/kerasy/engine/sequential.py:67: UserWarning: Kerasy Warnings
------------------------------------------------------------
When calculating the CategoricalCrossentropy loss and the derivative of the Softmax layer, the gradient disappears when backpropagating the actual value, so the SoftmaxCategoricalCrossentropy is implemented instead.
------------------------------------------------------------
  "so the \033SoftmaxCategoricalCrossentropy\033 is implemented instead.\n" + '-'*60)


In [11]:
model.summary()

-----------------------------------------------------------------
Layer (type)                 Output Shape              Param #   
input_1 (Input)              (None, 28, 28, 1)         0         
-----------------------------------------------------------------
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
-----------------------------------------------------------------
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
-----------------------------------------------------------------
maxpooling2d_1 (MaxPooling2D (None, 12, 12, 64)        0         
-----------------------------------------------------------------
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
-----------------------------------------------------------------
flatten_1 (Flatten)          (None, 9216)              0         
-----------------------------------------------------------------
dense_1 (Dense)              (None, 128)               1179776   
----------

In [12]:
model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(x_test, y_test)
)

Epoch 01/20 | 63/63[####################]100.00% - 1442.691[s]   categorical_crossentropy: 1381.319, categorical_accuracy: 54.4%, val_categorical_crossentropy: 740.757, val_categorical_accuracy: 78.9%
Epoch 02/20 | 63/63[####################]100.00% - 1439.734[s]   categorical_crossentropy: 573.693, categorical_accuracy: 83.7%, val_categorical_crossentropy: 584.554, val_categorical_accuracy: 86.2%
Epoch 03/20 | 63/63[####################]100.00% - 1318.618[s]   categorical_crossentropy: 405.320, categorical_accuracy: 88.2%, val_categorical_crossentropy: 418.508, val_categorical_accuracy: 90.4%
Epoch 04/20 | 63/63[####################]100.00% - 1010.807[s]   categorical_crossentropy: 318.465, categorical_accuracy: 90.0%, val_categorical_crossentropy: 440.983, val_categorical_accuracy: 90.1%
Epoch 05/20 | 63/63[####################]100.00% - 1020.449[s]   categorical_crossentropy: 261.625, categorical_accuracy: 92.6%, val_categorical_crossentropy: 496.641, val_categorical_accuracy: 89.6%

In [13]:
model.save_weights("MNIST_example_notebook_adagrad.pickle")